In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
batch_size = 64
epochs = 300
augmentations = transforms.Compose([transforms.RandomHorizontalFlip(),transforms.RandomCrop(32,padding=4),
                                   transforms.ToTensor(),transforms.Normalize((0.0,0.0,0.0),(1.0,1.0,1.0))])
test_tf = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.0,0.0,0.0),(1.0,1.0,1.0))])
trainset = torchvision.datasets.CIFAR10(root='./cifardata',train=True,download=True,transform=augmentations)
testset = torchvision.datasets.CIFAR10(root='./cifardata',train=False,download=True,transform=test_tf)
trainval = torchvision.datasets.CIFAR10(root='./cifardata',train=True,download=True,transform=test_tf)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=batch_size,shuffle=True,num_workers=4)
testloader = torch.utils.data.DataLoader(testset,batch_size=batch_size,shuffle=False,num_workers=4)
evalloader = torch.utils.data.DataLoader(trainval,batch_size=batch_size,shuffle=False,num_workers=4)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
class DenseUnit(nn.Module):
    def __init__(self,inchannels,growthrate):
        super(DenseUnit,self).__init__()
        self.conv = nn.Conv2d(inchannels,growthrate,3,padding=1,bias=False)
        self.bn = nn.BatchNorm2d(inchannels)
        self.drop = nn.Dropout(p=0.2)
    
    def forward(self,x):
        out = F.relu(self.bn(x))
        out = self.conv(out)
        out = self.drop(out)
        out = torch.cat([x,out],1)
        return out
    
class Transition(nn.Module):
    def __init__(self,inchannels,outchannels):
        super(Transition,self).__init__()
        self.conv = nn.Conv2d(inchannels,outchannels,1,bias=False)
        self.bn = nn.BatchNorm2d(inchannels)
        self.drop = nn.Dropout(p=0.2)
        self.avgpool = nn.AvgPool2d(2,stride=2)
    
    def forward(self,x):
        out = F.relu(self.bn(x))
        out = self.conv(out)
        out = self.drop(out)
        out = self.avgpool(out)
        return out

class DenseNet(nn.Module):
    def __init__(self,growthrate,n_classes):
        super(DenseNet,self).__init__()
        n_channels = 16
        self.conv = nn.Conv2d(3,n_channels,3,padding=1,bias=False)
        self.block1 = self.dense_block(12,n_channels,growthrate)
        n_channels += 12 * growthrate
        self.t12 = Transition(n_channels,160)
        n_channels = 160
        self.block2 = self.dense_block(12,n_channels,growthrate)
        n_channels += 12 * growthrate
        self.t23 = Transition(n_channels,304)
        n_channels = 304
        self.block3 = self.dense_block(12,n_channels,growthrate)
        n_channels += 12 * growthrate
        self.bn = nn.BatchNorm2d(n_channels)
        self.fc = nn.Linear(n_channels,n_classes)
        
        for m in self.modules():
            if isinstance(m,nn.Conv2d):
                nn.init.kaiming_normal_(m.weight,mode='fan_out',nonlinearity='relu')
            elif isinstance(m,nn.Linear):
                nn.init.xavier_normal_(m.weight)

    def forward(self,x):
        x = self.conv(x)
        x = self.block1(x)
        x = self.t12(x)
        x = self.block2(x)
        x = self.t23(x)
        x = self.block3(x)
        x = F.relu(self.bn(x))
        x = F.avg_pool2d(x,x.size(2))
        x = x.view(-1,x.size(1))
        x = self.fc(x)
        return x
    
    def dense_block(self,n_units,n_channels,growthrate):
        layers = []
        for i in range(n_units):
            layers.append(DenseUnit(n_channels,growthrate))
            n_channels += growthrate
        return nn.Sequential(*layers) 

In [4]:
densenet = DenseNet(12,10)
device = torch.device('cuda')
densenet.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(densenet.parameters(),lr=0.1,momentum=0.9,weight_decay=0.0001,nesterov=True) 
scheduler = optim.lr_scheduler.MultiStepLR(optimizer,milestones=[150,225])

In [5]:
def check_accuracy(loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            inputs,labels = data
            inputs,labels = inputs.to(device),labels.to(device)
            outputs = densenet(inputs)
            _,predicted = torch.max(outputs.data,1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return (correct/total)

In [6]:
losses = []
trains = []
evals = []
for epoch in range(300):
    running_loss = 0.0
    scheduler.step()
    for i,data in enumerate(trainloader,1):
        inputs,labels = data
        inputs,labels = inputs.to(device),labels.to(device)
        optimizer.zero_grad()
        loss = criterion(densenet(inputs),labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i%100==0:
            print('[Epoch %d,Minibatch %d] Average Training Loss : %.3f' % (epoch+1,i,running_loss/100))
            losses.append(running_loss)
            running_loss = 0
            
    trainaccuracy = check_accuracy(evalloader)
    evalaccuracy = check_accuracy(testloader)
    print('[Epoch %d] : Training Accuracy : %.3f Test Accuracy %.3f' % (epoch+1,trainaccuracy,evalaccuracy))
    trains.append(trainaccuracy)
    evals.append(evalaccuracy)

print("TRAINING IS NOW COMPLETE")

[Epoch 1,Minibatch 100] Average Training Loss : 1.929
[Epoch 1,Minibatch 200] Average Training Loss : 1.710
[Epoch 1,Minibatch 300] Average Training Loss : 1.561
[Epoch 1,Minibatch 400] Average Training Loss : 1.454
[Epoch 1,Minibatch 500] Average Training Loss : 1.387
[Epoch 1,Minibatch 600] Average Training Loss : 1.306
[Epoch 1,Minibatch 700] Average Training Loss : 1.229
[Epoch 1] : Training Accuracy : 0.572 Test Accuracy 0.562
[Epoch 2,Minibatch 100] Average Training Loss : 1.182
[Epoch 2,Minibatch 200] Average Training Loss : 1.111
[Epoch 2,Minibatch 300] Average Training Loss : 1.099
[Epoch 2,Minibatch 400] Average Training Loss : 1.056
[Epoch 2,Minibatch 500] Average Training Loss : 1.052
[Epoch 2,Minibatch 600] Average Training Loss : 1.016
[Epoch 2,Minibatch 700] Average Training Loss : 1.016
[Epoch 2] : Training Accuracy : 0.646 Test Accuracy 0.636
[Epoch 3,Minibatch 100] Average Training Loss : 0.961
[Epoch 3,Minibatch 200] Average Training Loss : 0.960
[Epoch 3,Minibatch 3

Process Process-482:
Process Process-483:
Process Process-481:
Process Process-484:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.6/dist-pack

KeyboardInterrupt: 

In [ ]:
plt.plot(trains,'r')
plt.plot(evals,'b')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
plt.plot(losses,'b')
plt.ylabel('Running Loss')
plt.show()

In [ ]:
torch.save(resnet.state_dict(),'DenseNet40')